# Chuleta rápida: statsmodels (OLS)
Breve guía con importaciones, uso básico, diagnóstico y opciones comunes. Código en Python, explicaciones mínimas y directas.

In [ ]:
# Imports esenciales
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from statsmodels.stats.stattools import durbin_watson
# para gráficos rápidos (opcional)
import matplotlib.pyplot as plt
plt.style.use('seaborn')

## Cargar ejemplo y ajustar OLS (API de fórmulas)

In [ ]:
# ejemplo con dataset mtcars incluido en statsmodels
df = sm.datasets.get_rdataset('mtcars').data
df.head()  # inspección rápida

# Ajuste con formula API (fácil y legible)
mod = smf.ols('mpg ~ wt + hp', data=df).fit()
print(mod.summary())

# SE robustas (HC3):
print(mod.get_robustcov_results(cov_type='HC3').summary())

## OLS con API clásica (Matriz X, añadir constante)

In [ ]:
X = df[['wt', 'hp']]
X = sm.add_constant(X)  # intercepto
y = df['mpg']
ols = sm.OLS(y, X).fit()
ols.summary()

## Diagnóstico básico y estadísticas útiles

In [ ]:
resid = ols.resid
fitted = ols.fittedvalues

# QQ-plot residuos (normalidad)
qqplot(resid, line='s'); plt.title('QQ plot residuos')

# Heterocedasticidad: Breusch-Pagan
bp_test = het_breuschpagan(resid, ols.model.exog)
# devuelve (Lagrange multiplier stat, p-value, f-stat, f p-value)
print('Breusch-Pagan p-value:', bp_test[1])

# White test
white_test = het_white(resid, ols.model.exog)
print('White test p-value:', white_test[1])

# Autocorrelación: Durbin-Watson (1-4, ideal ~2)
print('Durbin-Watson:', durbin_watson(resid))

# Influencia / leverage / Cook's distance
inf = OLSInfluence(ols)
leverage = inf.hat_matrix_diag
cooks_d = inf.cooks_distance[0]
# inspeccionar puntos influyentes (ejemplo: top 5 Cook's)
influential_idx = np.argsort(cooks_d)[-5:][::-1]
print('Top Cook''s distance indices:', influential_idx)

## Predicción y intervalos de confianza

In [ ]:
# ejemplo de predicción con conf_int
new = pd.DataFrame({'wt':[3.0, 2.5], 'hp':[110, 95]})
new = sm.add_constant(new)
pred = ols.get_prediction(new)
pred_summary = pred.summary_frame(alpha=0.05)  # mean, mean_se, mean_ci_lower/upper, obs_ci_lower/upper
pred_summary

## Modelos relacionados y opciones rápidas
- GLM: `sm.GLM(y, X, family=sm.families.Gaussian())` (otra familia: Poisson, Binomial)
- MixedLM: modelos lineales mixtos (`sm.MixedLM`) para efectos aleatorios
- Robust cov: `get_robustcov_results(cov_type='HC3')` o `cov_type='cluster'`
- Formula API soporta transformaciones: `y ~ np.log(x) + C(category)`

In [ ]:
# Ejemplos mínimos: GLM y MixedLM (esqueleto)
glm = sm.GLM(df['mpg'], sm.add_constant(df[['wt','hp']]), family=sm.families.Gaussian()).fit()
print(glm.summary())
# MixedLM (requiere datos jerárquicos):
# from statsmodels.regression.mixed_linear_model import MixedLM
# md = MixedLM(endog, exog, groups=group_var).fit()

## Notas rápidas (resumen)
- Use `smf.ols` para comodidad con fórmulas; `sm.OLS` cuando necesite control directo sobre matrices.
- Siempre verifique heterocedasticidad y autocorrelación; use SE robustas si hay dudas.
- Inspeccione influencia (Cook's), leverage y outliers antes de interpretar coeficientes.
- Consulte la documentación oficial: https://www.statsmodels.org/